In [16]:
!pip install rasterio tqdm datasets

In [17]:
import os
import numpy as np
import rasterio
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import json
from datasets import load_from_disk

In [3]:
!conda install -y gdown

warning  libmamba Cache file "/opt/conda/pkgs/cache/c7b05ae9.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/758dc0ed.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/eab6ca9b.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/6d94291a.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/3864cda3.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/47534735.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/88ec62ec.json" was modified by another program
warning  libmamba Cache file "/opt/conda/pkgs/cache/eb045dd1.json" was modified by another program
[+] 0.0s
[+] 0.1s
nodefaults/linux-64 (check zst) ━━━━╸━━━━━━━━━━━━━━━╸━━━━   0.0 B Checking  0.0s
nodefaults/noarch (check zst)   ━━━━━━━━━━━╸━━━━━━━━━━━━━   0.0 B Checking  0.0s
nvidia/linux-64 (check zst) 

In [6]:
!gdown 1SZgGMVEZC2xiR33uK8Viz6oxd0IealJF

Downloading...
From (original): https://drive.google.com/uc?id=1SZgGMVEZC2xiR33uK8Viz6oxd0IealJF
From (redirected): https://drive.google.com/uc?id=1SZgGMVEZC2xiR33uK8Viz6oxd0IealJF&confirm=t&uuid=ae11a081-9f51-47f5-afac-01c7d6c43940
To: /kaggle/working/multiclass_dataset.zip
100%|██████████████████████████████████████| 2.91G/2.91G [00:41<00:00, 70.2MB/s]


In [7]:
!unzip "/kaggle/working/multiclass_dataset.zip" -d "/kaggle/working/multiclass_dataset"

Archive:  /kaggle/working/multiclass_dataset.zip
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_0_1.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_0_2.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_0_3.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_0_4.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_0_5.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_0_6.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_0_7.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_0_8.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_1_0.tif  
  inflating: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_1_1.tif  
  inflating: /kaggle/working/multiclass_dataset/inp

In [11]:
base_dir = "/kaggle/working/multiclass_dataset"

input_dir = os.path.join(base_dir, "input_tiles")

if not os.path.exists(input_dir):
    print(f"Input directory not found for")

for file_name in os.listdir(input_dir):
    if "output_" in file_name:
        old_path = os.path.join(input_dir, file_name)
        new_file_name = file_name.replace("output_", "input_")
        new_path = os.path.join(input_dir, new_file_name)
        
        os.rename(old_path, new_path)
        print(f"Renamed: {old_path} -> {new_path}")
    else:
        print(f"No 'output_' found in: {file_name}")

Renamed: /kaggle/working/multiclass_dataset/input_tiles/amora_output_tile_2_0.tif -> /kaggle/working/multiclass_dataset/input_tiles/amora_input_tile_2_0.tif
Renamed: /kaggle/working/multiclass_dataset/input_tiles/suragpur_output_tile_8_3.tif -> /kaggle/working/multiclass_dataset/input_tiles/suragpur_input_tile_8_3.tif
Renamed: /kaggle/working/multiclass_dataset/input_tiles/suragpur_output_tile_16_0.tif -> /kaggle/working/multiclass_dataset/input_tiles/suragpur_input_tile_16_0.tif
Renamed: /kaggle/working/multiclass_dataset/input_tiles/suragpur_output_tile_16_8.tif -> /kaggle/working/multiclass_dataset/input_tiles/suragpur_input_tile_16_8.tif
Renamed: /kaggle/working/multiclass_dataset/input_tiles/uplarshi_output_tile_0_4.tif -> /kaggle/working/multiclass_dataset/input_tiles/uplarshi_input_tile_0_4.tif
Renamed: /kaggle/working/multiclass_dataset/input_tiles/amirpur_output_tile_8_2.tif -> /kaggle/working/multiclass_dataset/input_tiles/amirpur_input_tile_8_2.tif
Renamed: /kaggle/working/m

In [12]:
data_dir = "/kaggle/working/multiclass_dataset"
categories = {"roads": 1, "waterbodies": 2, "roofs": 3, "other": 4}

In [14]:
processed_data = []
error_files = []
batch_size = 5

In [18]:
color_to_label = {
    (255, 0, 0): 1,    # Roads (Red)
    (0, 0, 255): 2,    # Water Bodies (Blue)
    (0, 128, 0): 3,    # RCC Rooftop (Green)
    (255, 255, 0): 4,  # Tin Rooftop (Yellow)
    (255, 0, 255): 5,  # Tiled Rooftop (Magenta)
    (165, 42, 42): 6   # Additional Class
}

def map_colors_to_labels(segmentation_map, color_to_label):
    """
    Map an RGB segmentation map to label values based on color mapping.

    Parameters:
    - segmentation_map (np.ndarray): The segmentation map (height x width x 3).
    - color_to_label (dict): Mapping from RGB tuples to label IDs.

    Returns:
    - labeled_segmentation_map (np.ndarray): Labeled map (height x width).
    """
    labeled_map = np.zeros((segmentation_map.shape[0], segmentation_map.shape[1]), dtype=np.uint8)
    for color, label in color_to_label.items():
        mask = np.all(segmentation_map == color, axis=-1)
        labeled_map[mask] = label
    return labeled_map

# Directories
input_dir = os.path.join(data_dir, "input_tiles")
output_dir = os.path.join(data_dir, "output_tiles")

file_names = [f for f in os.listdir(input_dir) if f.endswith(".tif")]
processed_data = []
error_files = []
batch_size = 10  # Define batch size

for i in range(0, len(file_names), batch_size):
    batch_files = file_names[i:i + batch_size]
    for file_name in tqdm(batch_files, desc="Processing files"):
        try:
            # Process input file
            input_path = os.path.join(input_dir, file_name)
            with rasterio.open(input_path) as src:
                input_image = src.read()
                input_image = np.transpose(input_image, (1, 2, 0))  # Convert to HWC
                input_image = input_image / 255.0  # Normalize to [0, 1]

            # Process output file
            output_file_name = file_name.replace("input_", "output_")
            output_path = os.path.join(output_dir, output_file_name)
            with rasterio.open(output_path) as src:
                segmentation_map = src.read()
                segmentation_map = np.transpose(segmentation_map, (1, 2, 0))  # Convert to HWC

                labeled_segmentation_map = map_colors_to_labels(segmentation_map, color_to_label)

            processed_data.append({
                "pixel_values": input_image,
                "label": labeled_segmentation_map,
            })
        except Exception as e:
            error_files.append(file_name)
            print(f"Error processing file {file_name}: {e}")

    dataset = Dataset.from_dict({
        "pixel_values": [d["pixel_values"] for d in processed_data],
        "label": [d["label"] for d in processed_data]
    })
    dataset.save_to_disk(f"processed_hf_dataset_batch_{i // batch_size}")
    processed_data.clear()

if error_files:
    print(f"Files with errors: {error_files}")

Processing files: 100%|██████████| 10/10 [00:03<00:00,  2.52it/s]


Saving the dataset (0/7 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Processing files: 100%|██████████| 10/10 [00:07<00:00,  1.31it/s]


Saving the dataset (0/7 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Processing files: 100%|██████████| 10/10 [00:05<00:00,  1.70it/s]


Saving the dataset (0/7 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

OSError: [Errno 28] No space left on device

In [ ]:
parent_dir = os.getcwd()

dataset_paths = [
    os.path.join(parent_dir, d) for d in os.listdir(parent_dir)
    if os.path.isdir(os.path.join(parent_dir, d)) and "processed_hf_dataset" in d
]

datasets = [load_from_disk(path) for path in dataset_paths]

print(f"Found datasets: {dataset_paths}")

In [ ]:
combined_dataset = concatenate_datasets(datasets)

In [ ]:
dataset = Dataset.from_dict({"pixel_values": [d["pixel_values"] for d in processed_data],
                             "label": [d["label"] for d in processed_data]})

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_val = dataset["train"].train_test_split(test_size=0.1, seed=42)

In [ ]:
dataset_dict = DatasetDict({
    "train": train_val["train"],
    "validation": train_val["test"],
    "test": dataset["test"]
})

In [ ]:
dataset_dict.save_to_disk("combined_hf_dataset_dict")

In [ ]:
id2label = {0: "roads", 1: "waterbodies", 2: "roofs"}
with open("combined_hf_dataset_dict/id2label.json", "w") as f:
    json.dump(id2label, f, indent=4)
print("id2label.json created.")

In [ ]:
dataset = load_from_disk("processed_hf_dataset")
dataset.push_to_hub("itsskofficial/sih-data")
print("Dataset uploaded to Hugging Face.")